# Implementing UNet Architecture From Scratch Using PyTorch

In [ ]:
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as T
import torchvision as TV
import matplotlib.pyplot as plt